This notebook combines the proton dose DICOM files from the 4 Topas simulations into one single image. It can also scale the dose by the clinical RBE of 1.1

In [ ]:
#Install the Pydiocom Library
!pip install pydicom

     |████████████████████████████████| 1.9MB 6.7MB/s 


In [ ]:
#Import the pydicom library to access the data from dicom files
import pydicom
import numpy as np

In [ ]:
#Open the Proton Dose Files
PDoseBase = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonDose_Beam40_S1.dcm")
PDose40  = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonDose_Beam40_S1.dcm")
PDose160 = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonDose_Beam160_S1.dcm")
PDose200 = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonDose_Beam200_S1.dcm")
PDose320 = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonDose_Beam320_S1.dcm")

In [ ]:
#This section creates a combined DICOM of all 4 beams

#Use this file as a base for the new information to be written on
PDosearr = PDoseBase.pixel_array
#Select a dose grid scale
PD_scale = 10**(-9)

#Iterate over each voxel and combine the dose of all four beams
for z in range(PDoseBase.NumberOfFrames):
  for x in range(PDoseBase.Rows):
    for y in range(PDoseBase.Columns):
      PDosearr[z][x][y] = (1/PD_scale)*1000*(PDose40.pixel_array[z][x][y]*PDose40.DoseGridScaling +
                                           PDose160.pixel_array[z][x][y]*PDose160.DoseGridScaling + 
                                           PDose200.pixel_array[z][x][y]*PDose200.DoseGridScaling + 
                                           PDose320.pixel_array[z][x][y]*PDose320.DoseGridScaling)

#Save the output data
PDoseBase.PixelData = PDosearr.tobytes()     
PDoseBase.save_as("/content/drive/MyDrive/ProtonDoseLET/ProtonOneFrac.dcm")

In [ ]:
#Open the combined DICOM image
pro_base = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonOneFrac.dcm")
pro_1f = pydicom.dcmread("/content/drive/MyDrive/ProtonDoseLET/ProtonOneFrac.dcm")

arr11 = pro_base.pixel_array
PD_scale = 10**(-9)

#Iterate over each voxel and multiply the dose by a RBE of 1.1.
for z in range(pro_base.NumberOfFrames):
  for x in range(pro_base.Rows):
    for y in range(pro_base.Columns):
      arr11[z][x][y]= 1.1 * pro_1f.pixel_array[z][x][y]

#Save the DICOM
pro_base.PixelData = arr11.tobytes()     
pro_base.save_as("/content/drive/MyDrive/RBEscale/ClinScale.dcm")